<a href="https://colab.research.google.com/github/Alenushka2013/DataCamp-track-ML-scientist-in-Python/blob/main/2_project_Predictive_Modeling_for_Agriculture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called soil_measures.csv, which contains:

"N": Nitrogen content ratio in the soil

"P": Phosphorous content ratio in the soil

"K": Potassium content ratio in the soil

"pH" value of the soil

"crop": categorical values that contain various crops (target variable).
Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the "crop" column is the optimal choice for that field.

In this project, you will build multi-class classification models to predict the type of "crop" and identify the single most importance feature for predictive performance.


Оскільки у вас 22 класи, Логістична Регресія будуватиме модель або за схемою **One-vs-Rest (OvR)** (22 окремі моделі), або **Multinomial** (Softmax). У будь-якому випадку, інтерпретація важливості трохи складніша, ніж у бінарній класифікації, але цілком можлива.

Ось три найкращі підходи для визначення **однієї найважливішої ознаки**, від найпростішого до найбільш точного.

### Важливий нюанс перед початком
Для Логістичної Регресії **критично важливо масштабувати дані** (StandardScaler). Якщо одна ознака вимірюється в мільйонах, а інша в десятках, модель штучно завищить або занизить коефіцієнти, і визначення важливості буде хибним.

---

### Підхід 1: "Чесний" тест (Univariate Performance)
Це найнадійніший спосіб відповісти на питання "Яка *одна* ознака дає найкращий прогноз?". Ми просто навчаємо модель 5 разів — кожен раз використовуючи лише одну ознаку, і порівнюємо точність (або F1-score).

### Підхід 2: Середнє абсолютне значення коефіцієнтів
Логістична регресія створює матрицю коефіцієнтів розміром `[22 класи x 5 ознак]`. Щоб знайти глобальну важливість ознаки, ми беремо модуль усіх її коефіцієнтів для всіх класів і знаходимо середнє значення.

### Підхід 3: Перестановочна важливість (Permutation Importance)
Ми навчаємо модель на всіх ознаках, а потім по черзі "перемішуємо" (shuffling) значення однієї ознаки, перетворюючи її на шум. Те, наскільки сильно впаде якість моделі, і є показником важливості.


### Рекомендація

Оскільки ваша цільова змінна має **22 класи**, а ознак всього 5:

1.  **Орієнтуйтесь на Метод 1 (Однофакторний аналіз).**
    Коефіцієнти в багатокласовій логістичній регресії можуть бути складними для інтерпретації (ознака може бути важливою для класу №1, але заважати класу №5). Метод прямого навчання на одній ознаці покаже вам реальну прогностичну силу.

2.  **Перевірте баланс класів.**
    Якщо у вас 22 класи, ймовірно, якісь із них зустрічаються рідко. Якщо класи незбалансовані, обов'язково використовуйте метрику `f1_macro` (як у моєму коді) або додайте параметр `class_weight='balanced'` у модель Логістичної Регресії.

3.  **Нелінійність.**
    Логістична регресія шукає лінійні залежності. Якщо "найважливіша" ознака має нелінійний зв'язок (наприклад, U-подібний), логістична регресія може її не помітити. Якщо результати будуть поганими, спробуйте замінити `LogisticRegression` на `RandomForestClassifier` у коді вище (він краще ловить нелінійність).

In [4]:
# All required libraries are imported here for you.
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("soil_measures.csv") # https://www.kaggle.com/datasets/ahmedashrafahmed/soil-measures

X = crops.drop('crop', axis = 1)
y = crops.crop

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression(solver = 'lbfgs', max_iter = 1000, random_state = 42)
feature_names = crops.columns[:-1]

print('Однофакторний аналіз  - навчання на одній ознаці')

single_feature_scores = {}

for i, col in enumerate(feature_names):
    X_single = X_scaled[:, i].reshape(-1, 1)

    scores = cross_val_score(model, X_single, y, cv=5, scoring='f1_macro')
    single_feature_scores[col] = scores.mean()

best_single_feature = max(single_feature_scores, key=single_feature_scores.get)
for feature, score in sorted(single_feature_scores.items(), key=lambda item: item[1], reverse=True):
    print(f"{feature}: F1-score = {score:.4f}")

print(f"\n🏆 Переможець: {best_single_feature}")

Однофакторний аналіз  - навчання на одній ознаці
K: F1-score = 0.1966
P: F1-score = 0.1502
N: F1-score = 0.1060
ph: F1-score = 0.0788

🏆 Переможець: K


In [3]:
print('Важливість за коефіцієнтами')

model.fit(X_scaled, y)

coef_importance = np.mean(np.abs(model.coef_), axis = 0)

coef_dict = dict(zip(feature_names, coef_importance))
sorted_coefs = sorted(coef_dict.items(), key=lambda item: item[1], reverse=True)

for feature, score in sorted_coefs:
    print(f"{feature}: Середній |coef| = {score:.4f}")

print(f"🏆 Переможець: {sorted_coefs[0][0]}")

Важливість за коефіцієнтами
K: Середній |coef| = 3.5303
P: Середній |coef| = 3.2847
N: Середній |coef| = 2.6328
ph: Середній |coef| = 0.6925
🏆 Переможець (Метод 2): K


Для задачі визначення важливості ознак (особливо при 22 класах і всього 5 ознаках), параметри `RandomForest` мають бути налаштовані так, щоб модель була **стабільною**, а не просто максимально точною.

Ось рекомендована конфігурація з поясненням кожного параметра:

### Рекомендовані параметри

```python
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=500,        # Більше дерев = точніша оцінка важливості
    max_depth=None,          # Не обмежуємо глибину, бо 22 класи вимагають глибоких дерев
    min_samples_leaf=5,      # Захист від шуму (щоб не вчити унікальні викиди)
    max_features='sqrt',     # Важливо для відбору ознак (пояснення нижче)
    class_weight='balanced', # Критично для 22 класів (якщо є дисбаланс)
    random_state=42,         # Для відтворюваності
    n_jobs=-1                # Використовувати всі ядра процесора
)
```

### Чому саме такі налаштування?

1.  **`n_estimators=500` (Кількість дерев)**
    *   *Стандарт:* 100.
    *   *Чому змінили:* Для простого прогнозу вистачить і 100. Але для **Feature Importance** (важливості ознак) потрібна статистика. Чим більше дерев, тим стабільнішим буде рейтинг ознак при кожному запуску.

2.  **`max_features='sqrt'` (Ознак на спліт)**
    *   Це стандартне значення, але тут воно критичне.
    *   У вас всього 5 ознак. `sqrt(5) ≈ 2.23`. Тобто кожне дерево при розбитті вузла буде вибирати випадкові 2 ознаки.
    *   *Навіщо:* Це дає шанс "слабшим" ознакам проявити себе. Якщо поставити `None` (брати всі 5 ознак), то найсильніша ознака завжди стоятиме в корені дерева, і ми не дізнаємось про ефективність інших.

3.  **`min_samples_leaf=5`**
    *   Забороняє створювати листки (кінцеві рішення), де менше 5 об'єктів.
    *   Це згладжує модель. Якщо ознака "важлива" тільки для 1-2 конкретних прикладів (шум), цей параметр зменшить її фіктивну важливість.

4.  **`class_weight='balanced'`**
    *   При 22 класах майже гарантовано, що одних прикладів більше, інших менше. Без цього параметра модель може ігнорувати рідкісні класи, і ознаки, які відповідають за них, отримають низьку важливість (хоча вони насправді важливі).


### Порада: Перевірте Permutation Importance
У Random Forest є вбудований метод `.feature_importances_` (на основі Gini impurity). Він класний, але має недолік: він іноді завищує важливість ознак, які мають дуже велику варіативність числових значень.

Якщо хочете бути впевнені на 100%, застосуйте **Permutation Importance** до вже навченого лісу:

Якщо і Логістична регресія, і Gini importance лісу, і Permutation importance вкажуть на одну й ту ж ознаку — ви знайшли "золото".

In [5]:
rf_model = RandomForestClassifier(
    n_estimators=500,
    min_samples_leaf=5,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X, y)

importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1] # Сортування від найбільшого до найменшого

print("Рейтинг важливості ознак (Random Forest):")
for f in range(X.shape[1]):
    print(f"{f+1}. {feature_names[indices[f]]}: {importances[indices[f]]:.4f}")

top_feature = feature_names[indices[0]]
print(f"\n🏆 Найважливіша ознака: {top_feature}")

Рейтинг важливості ознак (Random Forest):
1. K: 0.3472
2. P: 0.2667
3. N: 0.1997
4. ph: 0.1864

🏆 Найважливіша ознака: K


In [6]:
from sklearn.inspection import permutation_importance

# Виконуємо на тренувальному сеті (або валідаційному, якщо є)
result = permutation_importance(rf_model, X, y, n_repeats=10, random_state=42, n_jobs=-1)

# Отримуємо середнє падіння точності
perm_indices = result.importances_mean.argsort()[::-1]

print("\nРейтинг за Permutation Importance (Найнадійніший):")
for i in perm_indices:
    print(f"{feature_names[i]}: {result.importances_mean[i]:.4f}")


Рейтинг за Permutation Importance (Найнадійніший):
K: 0.5596
P: 0.4920
N: 0.4512
ph: 0.2321
